In [ ]:
import keras, os
import numpy as np
from models import model1, model2

### Hyperparameters

In [2]:
n_timesteps = 30
n_securities = 20
input_shape=((n_timesteps-1, n_securities))
embed_dim = 384
n_heads = 6
n_layers = 4
ff_dim = embed_dim*4

batch_size = 16
epochs = 200
lr = 1e-4

### Data Pipeline

In [3]:
def preprocess_dataset(data, n_timesteps):
    n_examples = data.shape[0]-n_timesteps
    examples = []
    for idx in range(n_examples):
        example = data[idx:idx+n_timesteps]
        examples.append(example)
    return np.array(examples)

class dataset(keras.utils.PyDataset):

    def __init__(
            self,
            batch_size,
            examples,
            **kwargs       
            ):

        super().__init__(**kwargs)
        self.batch_size = batch_size
        self.examples = examples
    
    def __len__(self):
        return self.examples.shape[0] // self.batch_size

    def __getitem__(self, idx):
        X = self.examples[idx:idx+self.batch_size,:-1]
        Y = self.examples[idx:idx+self.batch_size,1:]
        return X, Y


### Load Dataset

In [ ]:
# FIXME: this is a placeholder for real data
P = np.random.randn(1000, n_securities)

examples = preprocess_dataset(P, n_timesteps)
n_examples = len(examples)
print(f'Found {n_examples} examples')

train_ds = dataset(batch_size, examples[:n_examples*80//10])
val_ds = dataset(batch_size, examples[:n_examples*20//100])

### Build Model

In [5]:
def compile(model, checkpoint, lr):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=keras.losses.MeanSquaredError(),
        )    
    if checkpoint is not None:
        if os.path.exists(checkpoint):
            print(f'Loaded model checkpoint "{checkpoint}"')
            model = model.load_weights(checkpoint)
        else:
            print(f'New model "{checkpoint}"')

In [ ]:
m1 = model1(input_shape, n_layers, embed_dim, n_heads, ff_dim)
m1.summary()

m1_checkpoint = None #'./checkpoints/model1.weights.h5'
compile(m1, m1_checkpoint, lr)

### Train Model

In [ ]:
m1.fit( 
    x=train_ds, epochs=epochs, validation_data=val_ds,
    callbacks=
    [
        #keras.callbacks.ModelCheckpoint(filepath=m1_checkpoint, save_weights_only=True, monitor="val_loss", mode="min", save_best_only=True),
    ])